<font color=gray>Oracle Cloud Infrastructure Data Science Sample Notebook

Copyright (c) 2021 Oracle, Inc.  All rights reserved. <br>
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

# Deploying an XGBoost Model with Model Deployment 

In this tutorial we are going to prepare and save an xgboost model artifact using the `ADSModel` `prepare()` method and deploy the model as an HTTP endpoint.

## Pre-requisites to Running this Notebook 

* We recommend that you run this notebook in a notebook session using the **Data Science Conda Environment "General Machine Learning for CPU (v1.0)"** 
* You need access to the public internet
* Upgrade the current version of the OCI Python SDK (`oci`): 

In [ ]:
!pip install --upgrade oci

In [2]:
import oci
import ads
import json
import logging
import os
import tempfile
import warnings
from os import path
from ads.common.model import ADSModel
from ads.common.model_artifact import ModelArtifact
from ads.dataset.dataset_browser import DatasetBrowser
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import time

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)
warnings.filterwarnings('ignore')
ads.set_documentation_mode(False)

We're going to train a simple XGBoost classifier on the breast cancer dataset included in sklearn; 

In [3]:
# Train xgboost model
breast_cancer = DatasetBrowser.sklearn().open('breast_cancer').set_target("target")
train, test = breast_cancer.train_test_split(test_size=0.15)
xgb_clf = XGBClassifier().fit(train.X.values, train.y)
xgb_bin_model = ADSModel.from_estimator(xgb_clf)

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

[14:40:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Here we are using the "General Machine Learning for CPU" Data Science conda environment. Since we don't modify the conda environment we don't need to publish it. We can use "General Machine learning for CPU (v1.0)" for model deployment as well. Thus we'll set `data_science_env=True` when preparing the artifact with ADS. 

Here we are using the `prepare()` method on an `ADSModel` object. 

In [4]:
# Prepare the model artifact template
path_to_model_artifacts = "xboost_artifacts"
model_artifact = xgb_bin_model.prepare(path_to_model_artifacts,
                                       force_overwrite=True,
                                       data_sample=test,
                                       fn_artifact_files_included=False,
                                       data_science_env=True)

loop1:   0%|          | 0/5 [00:00<?, ?it/s]

The inference conda environment is None and the Python version is None.
Start loading model.onnx from model directory /Users/caolingxin/Documents/workspaces/oci-projects/oci-ai-ads/ads_samples/xboost_artifacts ...
Model is successfully loaded.
Start loading model.onnx from model directory /Users/caolingxin/Documents/workspaces/oci-projects/oci-ai-ads/ads_samples/xboost_artifacts ...
Model is successfully loaded.


Let's take a look at the artifact template files that ADS generated. 

In [5]:
# List the template files
print(f"Model Artifact Path: {path_to_model_artifacts}\n\nModel Artifact Files:")
for file in os.listdir(path_to_model_artifacts):
    if path.isdir(path.join(path_to_model_artifacts, file)):
        for file2 in os.listdir(path.join(path_to_model_artifacts, file)):
            print(path.join(file, file2))
    else:
        print(file)

Model Artifact Path: xboost_artifacts

Model Artifact Files:
.model-ignore
input_schema.json
runtime.yaml
output_schema.json
onnx_data_transformer.json
model.onnx
score.py


Let's test the artifact before saving it to the model catalog and verify that the predictions made on a sample data match what we expect

In [6]:
# Validate predicion
model_artifact.predict(data=test.X[:5], model=model_artifact.load_model())

{'prediction': ['benign', 'malignant', 'malignant', 'benign', 'benign']}

We can now save the model to the catalog: 

In [8]:
project_id = "ocid1.datascienceproject.oc1.ap-singapore-1.amaaaaaacuco5yqaflrupyqjvr7m7edjfyyp355tqjdb7dnzd2b23kty7waq" 
compartment_id = "ocid1.compartment.oc1..aaaaaaaayjcsmu5ii7ac3kncp5qlbsslaj7irtc3mo4oco22w7ucsiq3atmq"

ads.set_auth(auth="api_key", oci_config_location=oci.config.DEFAULT_LOCATION, profile="specialist2-4sdk")

mc_model = model_artifact.save(
    project_id=project_id, compartment_id=compartment_id, 
    display_name="XGB_model (Model Deployment Test)",
    description="Testing XGB_model Model Deployment",
    ignore_pending_changes=True)

loop1:   0%|          | 0/5 [00:00<?, ?it/s]

The model metadata, including its OCID value (`id`): 

In [9]:
# Print published model information
mc_model

<style type="text/css">
#T_3988f_row0_col0, #T_3988f_row1_col0, #T_3988f_row2_col0, #T_3988f_row3_col0, #T_3988f_row4_col0, #T_3988f_row5_col0, #T_3988f_row6_col0, #T_3988f_row7_col0, #T_3988f_row8_col0, #T_3988f_row9_col0, #T_3988f_row10_col0, #T_3988f_row11_col0, #T_3988f_row12_col0 {
  margin-left: 0px;
}
</style>
<table id="T_3988f_">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th class="col_heading level0 col0" ></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th id="T_3988f_level0_row0" class="row_heading level0 row0" >display_name</th>
      <td id="T_3988f_row0_col0" class="data row0 col0" >XGB_model (Model Deployment Test)</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row1" class="row_heading level0 row1" >description</th>
      <td id="T_3988f_row1_col0" class="data row1 col0" >Testing XGB_model Model Deployment</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row2" class="row_heading level0 row2" >freeform_tags</th>
      <td id="T_3988f_row2_col0" class="data row2 col0" >{}</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row3" class="row_heading level0 row3" >defined_tags</th>
      <td id="T_3988f_row3_col0" class="data row3 col0" >{'owner': {'owner': 'speciallist2.admin02'}}</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row4" class="row_heading level0 row4" >repository_url</th>
      <td id="T_3988f_row4_col0" class="data row4 col0" >https://github.com/lxcao/oci-ai-ads.git</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row5" class="row_heading level0 row5" >git_branch</th>
      <td id="T_3988f_row5_col0" class="data row5 col0" >master</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row6" class="row_heading level0 row6" >git_commit</th>
      <td id="T_3988f_row6_col0" class="data row6 col0" >d966c657877fb9b1179302981d51891e7a152ff4</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row7" class="row_heading level0 row7" >script_dir</th>
      <td id="T_3988f_row7_col0" class="data row7 col0" >/Users/caolingxin/Documents/workspaces/oci-projects/oci-ai-ads/ads_samples/xboost_artifacts</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row8" class="row_heading level0 row8" >training_script</th>
      <td id="T_3988f_row8_col0" class="data row8 col0" >None</td>
    </tr>
    <tr>
      <th id="T_3988f_level0_row9" class="row_heading level0 row9" >schema_input</th>
      <td id="T_3988f_row9_col0" class="data row9 col0" >{'schema': [{'dtype': 'float64', 'feature_type': 'Continuous', 'name': 'mean_radius', 'domain': {'values': 'Continuous', 'stats': {'count': 86.0, 'mean': 14.114, 'standard deviation': 3.521, 'sample minimum': 6.981, 'lower quartile': 11.688, 'median': 13.405, 'upper quartile': 15.96, 'sample maximum': 25.22, 'skew': 0.816}, 'constraints': []}, 'required': True, 'description': 'mean_radius', 'order': 0}, {'dtype': 'float64', 'feature_type': 'Continuous', 'name': 'mean_texture', 'domain': {'values': 'Continuous', 'stats': {'count': 86.0, 'mean': 19.498, 'standard deviation': 4.185, 'sample minimum': 10.94, 'lower quartile': 16.218, 'median': 19.32, 'upper quartile': 22.365, 'sample maximum': 29.33, 'skew': 0.125}, 'constraints': []}, 'required': True, 'description': 'mean_texture', 'order': 1}, {'dtype': 'float64', 'feature_type': 'Continuous', 'name': 'mean_perimeter', 'domain': {'values': 'Continuous', 'stats': {'count': 86.0, 'mean': 92.018, 'standard deviation': 24.706, 'sample minimum': 43.79, 'lower quartile': 75.225, 'median': 87.17, 'upper quartile': 105.175, 'sample maximum': 171.5, 'skew': 0.896}, 'constraints': []}, 'required': True, 'description': 'mean_perimeter', 'order': 2}, {'dtype': 'float64', 'feature_type': 'Continuous', 'name': 'mean_area', 'domain': {'values': 'Continuous', 'stats': {'count': 86.0, 'mean': 651.25, 'standard deviation': 339.543, 'sample minimum': 143.5, 'lower quartile': 419.725, 'median': 544.05, 'upper quartile': 794.0, 'sample maximum': 1878.0, 'skew': 1.309}, 'constraints': []}, 'required': True,

## Deploying the model with Model Deployment

We are ready to deploy `mc_model`. We are using the user principal (config+key) method of authentication. Alternatively you can use resource principal. 

In [10]:
# Getting OCI config information
oci_config = oci.config.from_file("~/.oci/config", "specialist2-4sdk")
# Setting up DataScience instance
data_science = oci.data_science.DataScienceClient(oci_config)
# Setting up data science composite client to unlock wait_for_state operations
data_science_composite = oci.data_science.DataScienceClientCompositeOperations(data_science)

The model deployment configuration object: 

In [11]:
# Prepareing model deployment data
model_deployment_details = {
    "displayName": "XGB model test - ONNX",
    "projectId": mc_model.project_id,
    "compartmentId": mc_model.compartment_id,
    "modelDeploymentConfigurationDetails": {
        "deploymentType": "SINGLE_MODEL",
        "modelConfigurationDetails": {
            "modelId": mc_model.id,
            "instanceConfiguration": {
                "instanceShapeName": "VM.Standard2.4"
            },
            "scalingPolicy": {
                "policyType": "FIXED_SIZE",
                "instanceCount": 2
            },
            "bandwidthMbps": 10
        }
    },
    "categoryLogDetails": None
}

We are now ready to deploy. This takes a few minutes to complete. 

In [12]:
%%time

model_deployment = data_science_composite.create_model_deployment_and_wait_for_state(model_deployment_details,
                                                                                     wait_for_states=["SUCCEEDED",
                                                                                                      "FAILED"])

CPU times: user 901 ms, sys: 133 ms, total: 1.03 s
Wall time: 14min 58s


Let's make sure our deployment was successful: 

In [13]:
print("Grabbing the model deployment ocid...")
model_deployment_data = json.loads(str(model_deployment.data))
model_deployment_id = model_deployment_data['resources'][0]['identifier']
print(f"Model deployment ocid: {model_deployment_id}")

# check if the model deployment was successful: 
assert model_deployment.status == 200, f"Model deployment issued an HTTP error code: {model_deployment.status}"

Grabbing the model deployment ocid...
Model deployment ocid: ocid1.datasciencemodeldeployment.oc1.ap-singapore-1.amaaaaaacuco5yqamv7spddd6xljm326zenzw3xuaevgcuvw6slbs6g63iga


If the model deployment was unsuccessful, we recommend that you follow the Troubleshooting guide in our service documentation. 

## Invoking the Model Deployment `/predict` Endpoint 

Lastly we want to invoke the `/predict` endpoint of the deployed model and make inferences on a batch of 

In [14]:
import requests
import oci
from oci.signer import Signer

Before you can execute the cell below, copy and paste the URI of your model deployment. You can find that value in the OCI console under the detail page of your model deployment. In the **Resources** menu of the detail page, click on **"Invoking Your Model"**. You will find the HTTP endpoint of the model. 

In [15]:
uri = f"https://modeldeployment.ap-singapore-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ap-singapore-1.amaaaaaacuco5yqamv7spddd6xljm326zenzw3xuaevgcuvw6slbs6g63iga/predict"
print(uri)

https://modeldeployment.ap-singapore-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ap-singapore-1.amaaaaaacuco5yqamv7spddd6xljm326zenzw3xuaevgcuvw6slbs6g63iga/predict


In [16]:
using_rps = False

# payload: 
input_data = train.X[:5].to_json()

if using_rps: # using resource principal:     
    auth = oci.auth.signers.get_resource_principals_signer()
else: # using config + key: 
    config = oci.config.from_file("~/.oci/config", "specialist2-4sdk") # replace with the location of your oci config file
    auth = Signer(
        tenancy=config['tenancy'],
        user=config['user'],
        fingerprint=config['fingerprint'],
        private_key_file_location=config['key_file'],
        pass_phrase=config['pass_phrase'])

In [17]:
%%time
    
# submit request to model endpoint: 
response = requests.post(uri, json=input_data, auth=auth)

CPU times: user 25.6 ms, sys: 10.8 ms, total: 36.4 ms
Wall time: 1.28 s


Let's take a look at the status code: 

In [18]:
response.status_code

404

and the model predictions: 

In [19]:
print(json.loads(response.content))

{'code': 'NotAuthorizedOrNotFound', 'message': 'Authorization failed or requested resource not found.'}
